Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
!pip install yfinance
!pip install yahoofinancials

In [2]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: idna<3,>=2.5 in c:\users\toshiba\anaconda3\lib\site-packages (from requests>=2.19.0->pandas-datareader) (2.10)



In [3]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
from numpy import mean
from numpy import std
import pandas
import pandas as pd
import yfinance as yf

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [4]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "TSLA"

In [5]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol,pandas.Timestamp('2019'),pandas.Timestamp('2021'))
df = df[df.columns[0:4]]
df = df[['Open','High','Low','Close']]
df = df.dropna()

[*********************100%***********************]  1 of 1 completed


In [6]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df['Close'].rolling(window=15).mean()
df['Open_1_Change'] = df['Open'].diff()
df =df.dropna()
X = df

In [7]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where((df.Close.shift(-1)-df['Close'])>=0,1,-1)

In [8]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train = X[:index]
X_test  = X[index:]
y_train = y[:index]
y_test  = y[index:]

In [9]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

log = LogisticRegression()
log = log.fit(X_train,y_train)

In [10]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame({'column':X.columns.values, 
              'b' :log.coef_.ravel()})

,column,b
0,Open,-0.039429
1,High,0.019491
2,Low,0.084793
3,Close,-0.059249
4,Close_15_Rolling,-0.002924
5,Open_1_Change,0.029595


In [11]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = log.predict_proba(X_test)
predictions = log.predict(X_test)

In [12]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_pred=predictions,y_true=y_test)

array([[16, 45],
       [24, 63]], dtype=int64)

In [13]:
print(metrics.classification_report(y_pred=predictions,y_true=y_test))

              precision    recall  f1-score   support

          -1       0.40      0.26      0.32        61
           1       0.58      0.72      0.65        87

    accuracy                           0.53       148
   macro avg       0.49      0.49      0.48       148
weighted avg       0.51      0.53      0.51       148



In [14]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.
log.score(X_test,y_test)

0.5337837837837838

In [31]:
cross_val = cross_val_score(log,X=X,y=y, cv=5 )
cross_val

array([0.55555556, 0.53535354, 0.48979592, 0.53061224, 0.56122449])

In [29]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".
cv = KFold(n_splits=10, random_state=1, shuffle=True)
cross_val = cross_val_score(log,X_test,y_test,scoring='accuracy', cv=cv , n_jobs=-1)
cross_val

array([0.46666667, 0.66666667, 0.53333333, 0.46666667, 0.4       ,
       0.53333333, 0.6       , 0.6       , 0.35714286, 0.57142857])

In [28]:
print('Accuracy: %5f (%5f)' % (mean(cross_val), std(cross_val)))

Accuracy: 0.519524 (0.091608)
